In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline as Spline

## N-body problem for the solar system

As an example of an N-body evolution let us consider the solar system.  We will simplify our life dramatically by

1. Not including relativistic effects or the finite size/shape of bodies (i.e. treating them as point sources).
2. Sticking to the major solar system bodies.
3. Doing a "brute force" computation of the accelerations.
4. Using a simple-minded, low order integrator.

This will still be enough to get the basic idea.

To begin with the equations of motion are simple.  If $x_j$ is the 3D, Cartesian position of body $j$ in the chosen frame (for us the Solar System Barycenter -- SSB) then
$$
  \ddot{x}_j = \sum_{i\ne j} \frac{G m_i (x_i-x_j)}{|x_i-x_j|^3}
$$
Note, if we replace $|x_i-x_j|^3$ in the denominator with a function that does not go to zero but only to a very small quantity, then the sum will automatically exclude the $i=j$ term and we don't need to consider it separately.

Let us first consider how to compute the accelerations given a vector of positions.

In [ ]:
def acceleration(mass,pos,G=6.673e-20,eps=1.):
    """Given
    mass: a numpy array of size (Nbodies) of the masses in kg.
    pos:  a numpy array of size (Nbodies,3) of Cartesian positions in km.
    compute the accelerations, in km/s^2 returned as a numpy array of
    size (Nbodies,3)."""
    Nbodies  = mass.size
    # Extract the three coordinates.
    xx,yy,zz = pos[:,0],pos[:,1],pos[:,2]
    # Compute the difference "matrix": x_i-x_j for each of the coordinates:
    dx,dy,dz = xx[:,None]-xx[None,:],yy[:,None]-yy[None,:],zz[:,None]-zz[None,:]
    # and hence the denminator in the acceleration.  The "eps**2" is
    # just to avoid an overflow when i==j:
    rinv3 = (dx**2 + dy**2 + dz**2 + eps**2)**(-1.5)
    acc   = np.zeros( (Nbodies,3) )
    # Note that this is an O(N^2) sum, so for very large N would be
    # "prohibitively expensive" and more clever techniques would be needed.
    acc[:,0] = np.dot(mass,dx*rinv3) * G
    acc[:,1] = np.dot(mass,dy*rinv3) * G 
    acc[:,2] = np.dot(mass,dz*rinv3) * G             
    return(acc)
    #

Now we need initial condition data and we need to choose an integration scheme.

We can get the initial conditions from [NASA JPLs "Horizons" ephereris](https://ssd.jpl.nasa.gov/horizons/app.html#/) by selecting "Vector Table" in the "Ephemeris type" field and then pulling out the major objects 1 by 1.

In [ ]:
# Order of bodies is: Sun, Jupiter, Saturn, Earth.
# Units kg, km and sec.
# Start time: A.D. 2024-Aug-29 00:00:00.0000 TDB
#
mass = [1988500e24,1.89818722e27,5.6834e26,5.97219e24]
pos  = [\
         [-9.850907191013237E+05,-6.531770607421407E+05, 2.883058593038481E+04],\
         [ 2.931640346264411E+08, 6.941170814877204E+08,-9.438253743775815E+06],\
         [ 1.397465951978320E+09,-3.662742460504637E+08,-4.927131341947623E+07],\
         [ 1.368387986000750E+08,-6.255108076152393E+07, 3.110502304611728E+04],\
       ]
vel  = [\
         [ 1.124786898374933E-02,-8.579895010531720E-03,-1.649143410591409E-04],\
         [-1.218378931561312E+01, 5.705344672453313E+00, 2.489014930593703E-01],\
         [ 1.912632825537668E+00, 9.324040527096212E+00,-2.382309230325435E-01],\
         [ 1.174218017143991E+01, 2.705450701133305E+01,-1.783775069394977E-03],\
       ]
#
mass = np.array(mass)
pos  = np.array(pos)
vel  = np.array(vel)

We have to be _slightly_ careful here, because our ICs (in particular the total momentum or CoM velocity) are for the system with all of the bodies.  If we keep only some of them then the total momentum is changed and the system will translate over time by some amount.  We could either (a) subtract a constant from all of the velocities to ensure $P_{\rm tot}=0$ or (b) simply plot positions relative to the sun.  We'll do the former.

In [ ]:
print("Init. total momentum: ",np.dot(mass,vel))
print("Init. CoM velocity:   ",np.dot(mass,vel)/np.sum(mass))
#
vel -= np.dot(mass,vel)/np.sum(mass)
#
print("Final total momentum: ",np.dot(mass,vel))
print("Final CoM velocity:   ",np.dot(mass,vel)/np.sum(mass))

For the integration let us choose the ["position only" Verlet scheme](https://en.wikipedia.org/wiki/Verlet_integration), since we won't actually need the velocities for now.  Starting from $x_{\rm init}$ and $\dot{x}_{\rm init}$ and writing $A(x)$ as the acceleration for positions $x$, we compute
$$
  x_{\rm old} = x_{\rm init}
  \quad , \quad
  x_{\rm cur} = x_{\rm init} + \dot{x}_{\rm init} \Delta t +
  \frac{1}{2} A(x_{\rm init}) (\Delta t)^2
$$
Given $x_{\rm old}$ and $x_{\rm cur}$ we compute
$$
  x_{\rm new} = 2 x_{\rm cur} - x_{\rm old} + A(x_{\rm cur}) (\Delta t)^2
$$
and then iterate.

In [ ]:
def evolution(mass,pos,vel,tend=3.156e+9,dt=1e4,dtsave=1e6):
    """Does the N-body evolution."""
    # Set the initial conditions.
    tt   = 0.0
    acc  = acceleration(mass,pos)
    xold = pos.copy()
    xcur = xold + vel*dt + 0.5*acc*dt**2
    tt  += dt
    # Now do the integration, saving points every so
    # often so that we can make plots and do analysis.
    tlast,tsave,xsave = tt,[],[]
    while tt<tend:
        # The N-body step, using Verlet integration.
        xnew  = 2*xcur - xold + acceleration(mass,xcur)*dt**2
        xold  = xcur
        xcur  = xnew
        tt   += dt
        # Check to see whether to save this position.
        if tt-tlast>dtsave:
            # Positions have been updated to "tt".
            tsave.append(tt)
            xsave.append(xcur[:,:])
            tlast = tt
    tsave = np.array(tsave)
    xsave = np.array(xsave)
    return( (tsave,xsave) )

Let's make a run and plot the orbits just to show that this is working.  The default ```tend``` is roughly a century (in seconds) so each planet should complete at least one orbit.

In [ ]:
tsave,xsave = evolution(mass,pos,vel,dtsave=5e6)
#
fig,ax = plt.subplots(1,1,figsize=(5,5))
#
for ibody in range(mass.size):
    col = 'C'+str(ibody)
    ax.plot(xsave[:,ibody,0]/1e9,xsave[:,ibody,1]/1e9,'o',mfc='None',color=col)
#
ax.set_xlabel(r'$x$ ($10^9$km)')
ax.set_ylabel(r'$y$ ($10^9$km)')

Obviously you could "spice this up" by e.g. animating it or plotting the orbits in 3D or adding more planets etc.  If you want to do very long integrations then I recommend you not use this simple integrator but something a bit more sophisticated.

Let's check to see how accurate the integration for the Earth's position is (within the context of our simplified model) by varying the time step.

In [ ]:
# Compare the positions of the bodies with
# larger and smaller time steps to see how
# the integration is converging.

Generate some evolutions with and without the major planets.

How much does the position of the Earth change?

Now let's look at how close Jupiter and Saturn are as a function of time.

In [ ]:
# Compute the distance between bodies "1" and "2" (i.e. Jupiter and Saturn)
# as a function of time.
dist = # SOMETHING!
#
yr = 3.15576e+07  # 1yr in seconds.
print("Distance will be ",Spline(tfull,dist)(50*yr)," km.")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))
# Plot the distance vs. time, with time in years.
ax.plot(tfull/yr,dist/1e9,'-',color='C0')
#
ax.set_xlabel(r'$t$ (yr)')
ax.set_ylabel(r'Jupiter-Saturn distance ($10^9$km)')

How much does the Sun move around?

The nearest star, Proxima Centauri, is about $4\times 10^{13}$ km away.  If we were to view the solar system from directly above (i.e. along the $+\widehat{z}$ axis) what angle would this "Sun wobble" subtend at this distance?

## Satellite motion

Suppose that on 29th August 2024 a small satellite of mass $1000\,$kg was discovered on an inbound trajectory towards the sun.  Imagine it was traveling at $\vec{v}=15\,\widehat{y}\,\mathrm{km}\,\mathrm{s}^{-1}$ and was observed to be at $\vec{r}=(4.35,-4.25,-0.045)\times 10^8$km.

How far from the Sun will this satellite be in 2040?  What major event will influence this?

# The End